In [1]:
import numpy as np
import pandas as pd
import json
import os
from neo4j import GraphDatabase
import cv2
from ultralytics import YOLO
from glob import glob
import ast

In [2]:
URI = "neo4j://localhost:7687"  
USERNAME = "neo4j"            
PASSWORD = "Tai123321"     
model = YOLO('yolov8x-oiv7.pt') 

In [3]:
def get_neo4j_driver():
    return GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

In [4]:
def query_images(sorted_classes):
    image_ids = []
    with get_neo4j_driver().session() as session:
                query = """
                    WITH $classes AS classes
                    MATCH (img:Image)
                    WHERE ALL(class IN classes WHERE class IN[(img)-[:HAS_A]->(cls:Class) | cls.name])
                    RETURN img.image_Id AS image_id
                """
                result = session.run(query, classes=list(sorted_classes))
                for record in result:
                    if record["image_id"] not in image_ids:
                        image_ids.append(record["image_id"])
    if image_ids:
        return image_ids
    else:
        return None

In [74]:
def query_images_any(sorted_classes):
    image_ids = []
    with get_neo4j_driver().session() as session:
                query = """
                    WITH $classes AS classes
                    MATCH (img:Image)
                    WHERE ANY(class IN classes WHERE class IN[(img)-[:HAS_A]->(cls:Class) | cls.name])
                    RETURN img.image_Id AS image_id
                """
                result = session.run(query, classes=list(sorted_classes))
                for record in result:
                    if record["image_id"] not in image_ids:
                        image_ids.append(record["image_id"])
    if image_ids:
        return image_ids
    else:
        return None

In [6]:
def query_images_for_single_class(class_name):
    image_ids = []
    with get_neo4j_driver().session() as session:
                query = """
                    MATCH (img:Image)-[:HAS_A]->(cls:Class)
                    WHERE cls.name = $class_name
                    RETURN img.image_Id AS image_id
                """
                result = session.run(query, class_name=class_name)
                for record in result:
                    if record["image_id"] not in image_ids:
                        image_ids.append(record["image_id"])
    if image_ids:
        return image_ids
    else:
        return None

In [7]:
print(len(query_images_any(['Person', 'Clothing', 'Human face'])))

43001


In [8]:
def detect_objects(image_path):
    cv2_image = cv2.imread(image_path)
    results = model(cv2_image)
    objects = []
    for track_id, box in enumerate(results[0].boxes):
            class_label = results[0].names[int(box.cls)]
            conf = box.conf
            if class_label not in objects:
                objects.append(class_label)

    return objects

In [19]:
test_dataset = glob('../../../Smaller_Test_Imgs_Set/*.jpg')
ground_truth = pd.read_csv('../../data_files/ground_truth_top_level.csv')
mapping = pd.read_csv('../../data_files/mapping.csv')
mapping_dict = dict(zip(mapping['Description'], mapping['Description_Num']))
print(ground_truth.head())

            ImageID                                        Description  \
0  0000253ea4ecbf19  ['Carnivore', 'Human leg', 'Mammal', 'Plant', ...   
1  0000271195f2c007  ['Human eye', 'Headphones', 'Boy', 'Person', '...   
2  0000286a5c6a3eb5  ['Human eye', 'Sunglasses', 'Shorts', 'Person'...   
3  00003bfccf5f36c2  ['Person', 'Lantern', 'Chair', 'Table', 'Tree'...   
4  000045257f66b9e2  ['Boy', 'Person', 'Cowboy hat', 'Hat', 'Fedora...   

                                   Top Level Classes  
0        ['Plant', 'Mammal', 'Animal', 'Human body']  
1  ['Headphones', 'Person', 'Human body', 'Clothi...  
2  ['Human body', 'Clothing', 'Person', 'Animal',...  
3  ['Window', 'Lantern', 'Furniture', 'Plant', 'B...  
4  ['Human body', 'Hat', 'Clothing', 'Trousers', ...  


In [7]:
objects_extract_pd = pd.DataFrame(columns=['ImageID', 'Description'])
for image in test_dataset:
    image_name = image.split('/')[-1]
    image_id = image_name.split('.')[0].split('\\')[1]
    print(image_id)
    objects = detect_objects(image)
    print(objects)
    objects_extract_pd.loc[len(objects_extract_pd)] = [image_id, objects]
    

objects_extract_pd.to_csv('../../data_files/objects_extract_smaller_set.csv', index=False)

NameError: name 'test_dataset' is not defined

In [8]:
def convert_obj_to_num(mapping_dict, obj_list):
    number_list = [mapping_dict.get(desc, -1) for desc in obj_list]

    return number_list

In [9]:
def is_sublist(A, B):
    if not A:
        return True
    if not B:
        return False
    if A[0] == B[0]:
        return is_sublist(A[1:], B[1:])
    return is_sublist(A, B[1:])

In [66]:
def find_matching_images(dataset, detected_objects):
    matching_images = []
    
    # ground_truth['Description'] = ground_truth['Description'].apply(lambda x: ast.literal_eval(x))
    for index, row in dataset.iterrows():
        Description = row['Top Level Classes']
        image_id = row['ImageID']

        flag = False

        # Check if lists contain same elements (regardless of order)
        for detected in detected_objects:
            if detected not in Description:
                flag = True
                break
        if not flag:
            matching_images.append(image_id)
    
    return matching_images

In [77]:
def find_matching_images_1(dataset, detected_objects):
    matching_images = []
    
    # ground_truth['Description'] = ground_truth['Description'].apply(lambda x: ast.literal_eval(x))
    for index, row in dataset.iterrows():
        Description = row['Description']
        image_id = row['ImageID']

        flag = False

        # Check if lists contain same elements (regardless of order)
        for detected in detected_objects:
            if detected not in Description:
                flag = True
                break
        if not flag:
            matching_images.append(image_id)
    
    return matching_images

In [50]:
small_dataset = pd.read_csv('../../data_files/objects_extract_smaller_set.csv')
parent_map = json.load(open('../../data_files/parent_map.json'))

parent_df = pd.DataFrame(list(parent_map.items()) ,columns=['Child', 'Parent'])

parent_df.to_csv('../../data_files/parent_map.csv', index=False)

In [51]:
small_dataset["Description"] = small_dataset["Description"].apply(lambda x: ast.literal_eval(x))

In [60]:
# Function to convert a class to its top-level parent
def get_top_parent(class_name, parent_map):
    current = class_name
    while current in parent_map and parent_map[current] is not None:
        current = parent_map[current]
    return current

# Convert the descriptions to top-level classes
def convert_to_top_level(descriptions, parent_map):
    result = []
    for desc_list in descriptions:
        top_level_desc = [get_top_parent(class_name, parent_map) for class_name in desc_list]
        # Remove duplicates after conversion
        top_level_desc = list(set(top_level_desc))
        result.append(top_level_desc)
    return result

# Convert descriptions to top-level classes
parent_dict = dict(zip(parent_df['Child'], parent_df['Parent']))
small_dataset['Top_level_description'] = convert_to_top_level(small_dataset['Description'], parent_dict)

In [61]:
small_dataset.to_csv('../../data_files/objects_extract_smaller_set_top_level.csv', index=False)

In [55]:
print(small_dataset.head())

            ImageID                                        Description  \
0  23e61c230f9b8e2e             [Helmet, Person, Human face, Clothing]   
1  23f1cebf6b90c610      [Human face, Man, Sunglasses, Glasses, Woman]   
2  23f7498f91b27387  [Coat, Bicycle helmet, Backpack, Accordion, Ba...   
3  240297eb6148e310                 [Woman, Human face, Glasses, Tree]   
4  240310d0098c4c7e                           [Man, Shirt, Human face]   

                               Top_level_description  
0   [Human body, Person, Clothing, Sports equipment]  
1                [Human body, Person, Personal care]  
2  [Human body, Drink, Plant, Coin, Bathroom acce...  
3         [Human body, Person, Personal care, Plant]  
4                     [Human body, Person, Clothing]  


In [69]:
find_matching_images(ground_truth, small_dataset['Top_level_description'][1])

['0000286a5c6a3eb5',
 '000045257f66b9e2',
 '00005bf623ff1ac2',
 '0000ee1cc1f4c792',
 '00010417d07870a7',
 '0005421c4b4f4093',
 '0008fba87dd4aed2',
 '000ae28755d2d20e',
 '000c9b15cd1dcd95',
 '000de2d23a112361',
 '000fdfc1385fe190',
 '0010bb7d164c897f',
 '0011665172b9a374',
 '00124bbb1feb52f7',
 '00143d27992ccf33',
 '00151a36023916f0',
 '001591b1e0fe4778',
 '0015c629a1c04a9c',
 '00175d2750d675ca',
 '001782e87d635c9b',
 '0019d4f468fd8488',
 '001b37a8d55d9d48',
 '001bc5cccfdd71ca',
 '001be2b9260fe5ec',
 '001cc045c0f1755b',
 '001ddaff421aed10',
 '0021315335da8a8f',
 '0021c515128a6b1b',
 '0022aa7327636c98',
 '00247a79c7cb58a2',
 '00251745f963f8bc',
 '00255392a087eceb',
 '00257faf22e2defb',
 '0025e81774d62645',
 '0025ececb061efbd',
 '0027a5a564c93330',
 '002abe8747e84c36',
 '002bc6d5c5f58487',
 '002d354583d075eb',
 '003416cb219feb1e',
 '0034bd401ead3239',
 '0036f054bde6b992',
 '003d369788269bb2',
 '003f718ccc51c843',
 '003fe681696956ca',
 '004315dc84c809ff',
 '0043dbedaa3b4d82',
 '00447e8a501

In [79]:
def precision_recall(dataset, detected_objects):
    matching_images = find_matching_images_1(dataset, detected_objects)
    results_img = query_images(detected_objects)
    print(f"Matching Images: {matching_images}")
    print(f"Results Images: {results_img}")
    true_positives = 0
    false_positives = 0
    if results_img:
        for img in results_img:
            if img in matching_images:
                true_positives += 1
            else:
                false_positives += 1
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / len(matching_images)
        return precision, recall
    else:
        return 0, 0
    
list_precision_1 = [] 
list_recall_1 = []
for index, row in small_dataset.iterrows():
    detected_objects = row['Description']
    precision, recall = precision_recall(ground_truth, detected_objects)
    list_precision_1.append(precision)
    list_recall_1.append(recall)
    print(f"Precision: {precision} Recall: {recall}")

average_precision = sum(list_precision_1) / len(list_precision_1)
average_recall = sum(list_recall_1) / len(list_recall_1)
print(f"Average Precision: {average_precision} Average Recall: {average_recall}")

Matching Images: ['00ed80da990286c8', '010eaed5f56c79e6', '011e431cdc1e4162', '017eb9b8935e83bc', '02025f7c00ef66de', '02c6da49dad3c0e6', '031df2f238968e62', '03702b8b0f326253', '038fe752ed842b9c', '03c2b27af1fd660e', '03e9b4731f427093', '03fa1054eaea89e0', '0409317d43e843fe', '040a2f9aa41d3baa', '04d29c724c0b12b6', '0546c82a35006579', '056a771203fac523', '05809c4181cfc348', '0620b65d3e2f54b3', '07524a82e955c6e4', '07be4b1f784a131e', '07c144c677f2e455', '08b1ca9b168c9b98', '09fa435fe0ce7da6', '0a4c344da9813939', '0ab18e23bfd458d4', '0ca09951d3a6b533', '0cfab7eee3f1d67a', '0d0e119f573403ce', '0e3ce0cc77f8ccae', '0ec18f89842f04f2', '0ec7aa5f3ef536e4', '0f03595f118eebd8', '0f6b1d2921077ada', '0f76efc38092dcbc', '1046350c65b3befa', '1193f93d860edbd8', '11ee47067f652370', '12867138b66dc9f4', '129686341e997b40', '12bdcbf209cb42d4', '12e8e988a217fbde', '12ec9225c35c7325', '1355fe7a05beeb70', '136f7ef7aa212b7b', '154533218665b34e', '15e9aaad89b11d55', '16765c4d112b616b', '1685ad24ecd54215', '1

In [76]:

def precision_recall(dataset, parent_convert_obj, detected_objects):
    matching_images = find_matching_images(dataset, parent_convert_obj)
    results_img = query_images(detected_objects)
    print(f"Matching Images: {matching_images}")
    print(f"Results Images: {results_img}")
    true_positives = 0
    false_positives = 0
    if results_img:
        for img in results_img:
            if img in matching_images:
                true_positives += 1
            else:
                false_positives += 1
        precision = true_positives / (true_positives + false_positives)
        recall = true_positives / len(matching_images)
        return precision, recall
    else:
        return 0, 0
    
list_precision = [] 
list_recall = []
for index, row in small_dataset.iterrows():
    parent_convert_objs = row['Top_level_description']
    detected_objects = row['Description']
    precision, recall = precision_recall(ground_truth, parent_convert_objs, detected_objects)
    list_precision.append(precision)
    list_recall.append(recall)
    print(f"Precision: {precision} Recall: {recall}")

average_precision = sum(list_precision) / len(list_precision)
average_recall = sum(list_recall) / len(list_recall)
print(f"Average Precision: {average_precision} Average Recall: {average_recall}")

Matching Images: ['000f2708a72f9de2', '002107c2cdc1b568', '00ed80da990286c8', '010b49a298a85348', '010eaed5f56c79e6', '0114223477d4a23e', '011e431cdc1e4162', '0126fa247694f8d5', '017182fc784942ac', '017eb9b8935e83bc', '01e6f0a0584a2110', '02025f7c00ef66de', '021a5492cb605d70', '0220b847d05dd846', '025041a674592304', '02b72389c3338588', '02c6da49dad3c0e6', '031892fb53974e4c', '031df2f238968e62', '03702b8b0f326253', '038fe752ed842b9c', '03bdc2fe3683b092', '03c2b27af1fd660e', '03e9b4731f427093', '03fa1054eaea89e0', '0409317d43e843fe', '040a2f9aa41d3baa', '0455f98b9b49d786', '04c69b42375e742c', '04d29c724c0b12b6', '051e9a46514b9dc9', '0546c82a35006579', '056a771203fac523', '05809c4181cfc348', '0583a114dcce0256', '05f13d3980d5ddef', '0620b65d3e2f54b3', '064234cb569b0580', '066687387f5347f3', '072f39ae16cff3dc', '073c9a2479e71056', '07524a82e955c6e4', '07a2ff354d9ecf94', '07be4b1f784a131e', '07c144c677f2e455', '083e8094d9eb4a86', '0866633bbb8ee072', '08b1ca9b168c9b98', '090a358a1b139d65', '0

['00ed80da990286c8', '010eaed5f56c79e6', '011e431cdc1e4162', '017eb9b8935e83bc', '02025f7c00ef66de', '02c6da49dad3c0e6', '031df2f238968e62', '03702b8b0f326253', '038fe752ed842b9c', '03c2b27af1fd660e', '03e9b4731f427093', '03fa1054eaea89e0', '0409317d43e843fe', '040a2f9aa41d3baa', '04d29c724c0b12b6', '0546c82a35006579', '056a771203fac523', '05809c4181cfc348', '0620b65d3e2f54b3', '07524a82e955c6e4', '07be4b1f784a131e', '07c144c677f2e455', '08b1ca9b168c9b98', '09fa435fe0ce7da6', '0a4c344da9813939', '0ab18e23bfd458d4', '0ca09951d3a6b533', '0cfab7eee3f1d67a', '0d0e119f573403ce', '0e3ce0cc77f8ccae', '0ec18f89842f04f2', '0ec7aa5f3ef536e4', '0f03595f118eebd8', '0f6b1d2921077ada', '0f76efc38092dcbc', '1046350c65b3befa', '1193f93d860edbd8', '11ee47067f652370', '12867138b66dc9f4', '129686341e997b40', '12bdcbf209cb42d4', '12e8e988a217fbde', '12ec9225c35c7325', '1355fe7a05beeb70', '136f7ef7aa212b7b', '154533218665b34e', '15e9aaad89b11d55', '16765c4d112b616b', '1685ad24ecd54215', '16bf2596b61e46f0',

In [11]:
def find_matching_images_single_classes(dataset, classes):
    matching_images = []
    
    for index, row in dataset.iterrows():
        description = row['Description_Num']
        
        ground_truth = [int(x.strip()) for x in description[1:-1].split(",")]
        image_id = row['ImageID']

        class_number = mapping_dict.get(classes, -1)

        # Check if lists contain same elements (regardless of order)
        if class_number in ground_truth:
            matching_images.append(image_id)
    
    return matching_images

In [12]:
def find_all_matching_images(dataset, detected_objects):
    matching_images = []
    
    for index, row in dataset.iterrows():
        description = row['Description_Num']
        
        ground_truth = [int(x.strip()) for x in description[1:-1].split(",")]
        temp_detected_objects = ast.literal_eval(detected_objects)
        image_id = row['ImageID']

        number_list = convert_obj_to_num(mapping_dict, temp_detected_objects)

        # Check if lists contain same elements (regardless of order)
        break_flag = False
        for detected in number_list:
            if detected not in ground_truth:
                break_flag = True
                break
        if not break_flag:
            matching_images.append(image_id)
    
    return matching_images

In [13]:
def extracted_objects(pd_to_extract, image_id):
    objects = pd_to_extract[pd_to_extract['ImageID'] == image_id]['Description'].values[0]
    return objects

In [17]:
test_data_extract = pd.read_csv('../../data_files/objects_extract_smaller_set.csv')
print(test_data_extract.head())

            ImageID                                        Description
0  23e61c230f9b8e2e     ['Helmet', 'Person', 'Human face', 'Clothing']
1  23f1cebf6b90c610  ['Human face', 'Man', 'Sunglasses', 'Glasses',...
2  23f7498f91b27387  ['Coat', 'Bicycle helmet', 'Backpack', 'Accord...
3  240297eb6148e310         ['Woman', 'Human face', 'Glasses', 'Tree']
4  240310d0098c4c7e                     ['Man', 'Shirt', 'Human face']


In [15]:
img_ids = []
temp_dataset_extract_objs = []
for id in test_dataset:
    img_ids.append(id.split('/')[-1].split('.')[0].split('\\')[1])
print(img_ids[0])
for img_id in img_ids:
    if img_id in test_data_extract['ImageID'].values:
        detected_objects = extracted_objects(test_data_extract, img_id)
        temp_dataset_extract_objs.append(detected_objects)
    
print(temp_dataset_extract_objs[0])
print(len(temp_dataset_extract_objs))


NameError: name 'test_dataset' is not defined

In [18]:
for img in test_data_extract['ImageID']:
    detected_objects = extracted_objects(test_data_extract, img)
    list_detected_objects = ast.literal_eval(detected_objects)
    print(list_detected_objects[0])

Helmet
Human face
Coat
Woman
Man
Man
Man
Boy
Clothing
Boot


In [18]:
def calculate_Precision_Recall(img_id):
    total_imgs = 0
    correct_imgs = 0
    wrong_imgs = 0

    correct_imgs_ids = []
    wrong_imgs_ids = []

    detected_objects = extracted_objects(test_data_extract, img_id)

    list_detected_objects = ast.literal_eval(detected_objects)

    total_imgs = find_all_matching_images(ground_truth, detected_objects)
    
    query_images_list = query_images(list_detected_objects)

    if not query_images_list:
        if total_imgs:
            return 0, 0, total_imgs, [], list_detected_objects
        else:
            return 1, 1, [], [], list_detected_objects
    
    if not total_imgs:
        return 0, 0 

    for img in query_images_list:
        if img in total_imgs:
            correct_imgs += 1
            correct_imgs_ids.append(img)
        else:
            wrong_imgs += 1
            wrong_imgs_ids.append(img)

    print("Total images: ", len(total_imgs))
    print("Total query images: ", len(query_images_list))
    print("Correct images: ", correct_imgs)
    print("Wrong images: ", wrong_imgs)
    # print("Total images ids: ", total_imgs)
    # print("Correct images ids: ", correct_imgs_ids)
    # print("Wrong images ids: ", wrong_imgs_ids)

    precision = correct_imgs / (correct_imgs + wrong_imgs)
    recall = correct_imgs / len(total_imgs)

    #Find all the images that contain the detected objects in the ground truth
    return precision, recall, total_imgs, wrong_imgs_ids, list_detected_objects

In [35]:
def calculate_Precision_Recall_for_Single_Class(ground_truth_input, class_name):
    total_imgs = 0
    correct_imgs = 0
    wrong_imgs = 0

    total_imgs = find_matching_images_single_classes(ground_truth_input, class_name)
    
    query_images_list = query_images_for_single_class(class_name)

    if total_imgs:
        print("Total images: ", len(total_imgs))
    else:
        print("None of the images contain the class: ", class_name)
    if query_images_list:
        print("Total query images: ", len(query_images_list))
    else:
        print("No images found for the class: ", class_name)

    if not query_images_list:
        if total_imgs:
            return 0, 0
        else:
            return 1, 1
    
    if not total_imgs:
        return 0, 0 

    for img in query_images_list:
        if img in total_imgs:
            correct_imgs += 1
        else:
            wrong_imgs += 1

    print("Correct images: ", correct_imgs)
    print("Wrong images: ", wrong_imgs)
    # print("Total images ids: ", total_imgs)
    # print("Correct images ids: ", correct_imgs_ids)
    # print("Wrong images ids: ", wrong_imgs_ids)

    precision = correct_imgs / (correct_imgs + wrong_imgs)
    recall = correct_imgs / len(total_imgs)

    #Find all the images that contain the detected objects in the ground truth
    return precision, recall

In [36]:
df1 = pd.DataFrame(columns=['Query Classes', 'Precision', 'Recall'])
classes_freqs = json.load(open('../../data_files/class_frequencies.json'))
list_classes = []
for classes in classes_freqs:
    list_classes.append(classes["Class"])
print(list_classes)

['Accordion', 'Adhesive tape', 'Aircraft', 'Airplane', 'Alarm clock', 'Alpaca', 'Ambulance', 'Animal', 'Ant', 'Antelope', 'Apple', 'Armadillo', 'Artichoke', 'Auto part', 'Axe', 'Backpack', 'Bagel', 'Baked goods', 'Balance beam', 'Ball', 'Balloon', 'Banana', 'Band-aid', 'Banjo', 'Barge', 'Barrel', 'Baseball bat', 'Baseball glove', 'Bat (Animal)', 'Bathroom accessory', 'Bathroom cabinet', 'Bathtub', 'Beaker', 'Bear', 'Bed', 'Bee', 'Beehive', 'Beer', 'Beetle', 'Bell pepper', 'Belt', 'Bench', 'Bicycle', 'Bicycle helmet', 'Bicycle wheel', 'Bidet', 'Billboard', 'Billiard table', 'Binoculars', 'Bird', 'Blender', 'Blue jay', 'Boat', 'Bomb', 'Book', 'Bookcase', 'Boot', 'Bottle', 'Bottle opener', 'Bow and arrow', 'Bowl', 'Bowling equipment', 'Box', 'Boy', 'Brassiere', 'Bread', 'Briefcase', 'Broccoli', 'Bronze sculpture', 'Brown bear', 'Building', 'Bull', 'Burrito', 'Bus', 'Bust', 'Butterfly', 'Cabbage', 'Cabinetry', 'Cake', 'Cake stand', 'Calculator', 'Camel', 'Camera', 'Can opener', 'Canary', '

In [26]:
img_ids = pd.read_csv('../../data_files/export.csv')
print(img_ids.head())

  entity          image_Id
0   node  1077a7f431ff3ab7
1   node  04ebdcef386f13eb
2   node  434b1b05d650689d
3   node  31705f96656f6e59
4   node  2aa5338cb8ab54a5


In [29]:
img_ids = img_ids['image_Id'].values

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [30]:
print(img_ids[0])

1077a7f431ff3ab7


In [31]:
temp_ground_truth = ground_truth[ground_truth['ImageID'].isin(img_ids)]
print(len(temp_ground_truth))
print(len(ground_truth))

97273
105185


In [37]:
single_class_precision = []
single_class_recall = []
i = 0
for classes in list_classes:
    print(i)
    print("Class: ", classes)
    precision, recall = calculate_Precision_Recall_for_Single_Class(temp_ground_truth, classes)
    single_class_precision.append(precision)
    single_class_recall.append(recall)
    df1.loc[len(df1)] = [classes, precision, recall]
    print("Precision: ", precision)
    print("Recall: ", recall)
    i += 1
    
avg_precision = sum(single_class_precision) / len(single_class_precision)
avg_recall = sum(single_class_recall) / len(single_class_recall)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
df1.loc[len(df1)] = ['Average', avg_precision, avg_recall]
df1.to_csv('./precision_recall_single_class.csv', index=False)

0
Class:  Accordion
Total images:  39
Total query images:  49
Correct images:  37
Wrong images:  12
Precision:  0.7551020408163265
Recall:  0.9487179487179487
1
Class:  Adhesive tape
Total images:  12
Total query images:  2
Correct images:  2
Wrong images:  0
Precision:  1.0
Recall:  0.16666666666666666
2
Class:  Aircraft
Total images:  747
No images found for the class:  Aircraft
Precision:  0
Recall:  0
3
Class:  Airplane
Total images:  644
Total query images:  742
Correct images:  602
Wrong images:  140
Precision:  0.8113207547169812
Recall:  0.9347826086956522
4
Class:  Alarm clock
Total images:  9
Total query images:  3
Correct images:  2
Wrong images:  1
Precision:  0.6666666666666666
Recall:  0.2222222222222222
5
Class:  Alpaca
Total images:  30
Total query images:  32
Correct images:  25
Wrong images:  7
Precision:  0.78125
Recall:  0.8333333333333334
6
Class:  Ambulance
Total images:  19
Total query images:  4
Correct images:  3
Wrong images:  1
Precision:  0.75
Recall:  0.157

In [68]:
df = pd.DataFrame(columns=['Image', 'Query Classes', 'Precision', 'Recall', 'Correct Images', 'Wrong Images', 'Total Images'])
list_precision = []
list_recall = []
count = 0
for img in img_ids:
    if count == 2000:
        break

    if img in test_data_extract['ImageID'].values:
        print("Image: ", img)
        precision, recall, total_imgs, correct_imgs, wrong_imgs, list_detected_objects = calculate_Precision_Recall(img)
        print("Precision: ", precision)
        print("Recall: ", recall)
        print("----------------------------------------------------")
        list_precision.append(precision)
        list_recall.append(recall)

        df.loc[len(df)] = [img, list_detected_objects, precision, recall, correct_imgs, wrong_imgs, total_imgs]

        count += 1
        print(count)

df.loc[len(df)] = ["Average", [], sum(list_precision) / len(list_precision), sum(list_recall) / len(list_recall), [], [], []]
print("Average Precision: ", sum(list_precision) / len(list_precision))
print("Average Recall: ", sum(list_recall) / len(list_recall))
df.to_csv('../../data_files/Precision_Recall_smaller_1.csv', index=False)


Image:  23e61c230f9b8e2e
Total images:  167
Total query images:  29
Correct images:  14
Wrong images:  15
Precision:  0.4827586206896552
Recall:  0.08383233532934131
----------------------------------------------------
1
Image:  23f1cebf6b90c610
Total images:  85
Total query images:  102
Correct images:  41
Wrong images:  61
Precision:  0.4019607843137255
Recall:  0.4823529411764706
----------------------------------------------------
2
Image:  23f7498f91b27387
Precision:  1
Recall:  1
----------------------------------------------------
3
Image:  240297eb6148e310
Total images:  59
Total query images:  62
Correct images:  19
Wrong images:  43
Precision:  0.3064516129032258
Recall:  0.3220338983050847
----------------------------------------------------
4
Image:  240310d0098c4c7e
Total images:  150
Total query images:  204
Correct images:  61
Wrong images:  143
Precision:  0.29901960784313725
Recall:  0.4066666666666667
----------------------------------------------------
5
Image:  240c

In [72]:
df.loc[len(df)] = ["Average", sum(list_precision) / len(list_precision), sum(list_recall) / len(list_recall)]
print("Average Precision: ", sum(list_precision) / len(list_precision))
print("Average Recall: ", sum(list_recall) / len(list_recall))
df.to_csv('../../data_files/Precision_Recall.csv', index=False)

Average Precision:  0.49298485933943914
Average Recall:  0.4141173128903569


In [23]:
# Target objects we're looking for
target_objects = set(['Human face', 'Person', 'Clothing'])

# Load your ground truth data
# ground_truth_df = pd.read_csv('your_ground_truth_file.csv')

# Function to ensure we're working with a list
def ensure_list(description):
    if isinstance(description, str):
        try:
            # If it's a string representation of a list
            return ast.literal_eval(description)
        except:
            # If it's just a comma-separated string
            return [item.strip() for item in description.split(',')]
    return description  # Already a list

# Filter rows that contain all target objects
matching_rows = []
for idx, row in ground_truth.iterrows():
    description_list = ensure_list(row['Description'])
    # Check if all target objects are in the description
    if all(item in description_list for item in target_objects):
        matching_rows.append(row)

# Create DataFrame with matching rows
result_df = pd.DataFrame(matching_rows)

# Alternative one-liner using pandas
# result_df = ground_truth_df[ground_truth_df['Description'].apply(
#     lambda x: all(item in ensure_list(x) for item in target_objects))]

In [24]:
print(len(result_df))
print(result_df.head())

14837
             ImageID                                        Description  \
1   0000286a5c6a3eb5  ['Human eye', 'Sunglasses', 'Shorts', 'Person'...   
13  0001a2f47097ee27  ['Person', 'Necklace', 'Land vehicle', 'Earrin...   
26  00039573fc999278  ['Person', 'Coat', 'Human body', 'Woman', 'Hum...   
36  00059a3c8bfda05b  ['Toy', 'Person', 'Man', 'Clothing', 'Human fa...   
90  00107319659c65ba  ['Human eye', 'Dress', 'Person', 'Human mouth'...   

                                      Description_Num  
1   [264, 506, 457, 382, 271, 221, 270, 263, 267, ...  
13  [382, 352, 303, 178, 271, 263, 595, 267, 186, ...  
26                [382, 117, 262, 595, 269, 116, 265]  
36                          [548, 382, 323, 116, 265]  
90  [264, 170, 382, 271, 595, 267, 149, 186, 217, ...  


In [2]:
import json
import pandas as pd

In [4]:
bovw = json.load(open('../../data_files/bovw_pack(1).json'))
print(len(bovw))
print(bovw[0])

105185
{'ID': 'c52e4b7b42cd5427', 'Histogram': '[1.0, 0.29457244, 0.15516745, 0.26048854, 0.05792806, 0.026807811, -3.3589913e-09, 0.08570918, 1.0, 0.36185396, 0.2202106, 0.35515213, 0.083555646, 0.09081593, 5.8435035e-09, 0.0047498266, 1.0, 0.23618941, 0.2295012, 0.42980692, 0.2710422, 0.1482627, -1.0795702e-09, 0.03834669]', 'Objects': [{'Object_Id': 1, 'Class': 'Human face', 'Bbox': '[238.3320770263672, 256.9740295410156, 412.8487854003906, 434.0414123535156]', 'Conf': 0.8552523255348206}, {'Object_Id': 2, 'Class': 'Human face', 'Bbox': '[531.6046142578125, 161.68077087402344, 723.92041015625, 408.900146484375]', 'Conf': 0.8415064215660095}, {'Object_Id': 3, 'Class': 'Human face', 'Bbox': '[0.0, 86.51470184326172, 54.5564079284668, 233.65185546875]', 'Conf': 0.7054436802864075}, {'Object_Id': 4, 'Class': 'Woman', 'Bbox': '[0.0, 69.27558898925781, 240.3115692138672, 653.28955078125]', 'Conf': 0.6104355454444885}, {'Object_Id': 5, 'Class': 'Hat', 'Bbox': '[166.29629516601562, 30.09423

In [10]:
new_df = pd.DataFrame(columns=['ImageID', 'Description'])
for item in bovw:
    objects = []
    for object in item["Objects"]:
        if object["Class"] not in objects:
            objects.append(object["Class"])

    new_df.loc[len(new_df)] = [item["ID"], objects]
print(new_df.head())

            ImageID                              Description
0  c52e4b7b42cd5427  [Human face, Woman, Hat, Man, Clothing]
1  1c410e213a739937                         [Building, Tree]
2  d88015b72d112657             [Footwear, Woman, Miniskirt]
3  ee38e3f18badd0e8                               [Building]
4  e917fe6a7554ec39                                   [Tree]


In [11]:
new_df.to_csv('./bowv/new_database_bovw.csv', index=False)

In [13]:
print(len(new_df))
print(new_df.tail())

105185
                 ImageID                                        Description
105180  0f635c188f52276e                                        [Hamburger]
105181  0d1b92f6f3a1ab64                                    [Man, Clothing]
105182  95c62e010953cac4  [Coffee table, Chair, Kitchen & dining room ta...
105183  f79b787aa7856ec6                                        [Cosmetics]
105184  11aae6f3eceeb0a4                                    [Man, Clothing]


In [14]:
new_ground_truth = pd.read_csv('../../data_files/filtered_ground_truth.csv')

In [15]:
print(len(new_ground_truth))

105185


In [16]:
new_dataset_ground_truth_compare = pd.merge(new_df, new_ground_truth, on='ImageID', how='inner')
print(len(new_dataset_ground_truth_compare))

105185


In [17]:
print(new_dataset_ground_truth_compare.head())

            ImageID                            Description_x  \
0  c52e4b7b42cd5427  [Human face, Woman, Hat, Man, Clothing]   
1  1c410e213a739937                         [Building, Tree]   
2  d88015b72d112657             [Footwear, Woman, Miniskirt]   
3  ee38e3f18badd0e8                               [Building]   
4  e917fe6a7554ec39                                   [Tree]   

                                       Description_y  
0        ['Person', 'Hat', 'Clothing', 'Human face']  
1             ['House', 'Plant', 'Tree', 'Building']  
2  ['Shorts', 'Miniskirt', 'Person', 'Woman', 'Gi...  
3                                       ['Building']  
4                                           ['Tree']  


In [19]:
new_dataset_ground_truth_compare.to_csv('../../data_files/compare_ground_truth.csv', index=False)

In [20]:
df1 = pd.DataFrame(columns=['Query Classes', 'Precision', 'Recall', 'Wrong Images', 'Total Images'])
classes_freqs = json.load(open('../../data_files/class_frequencies.json'))
list_classes = []
for classes in classes_freqs:
    list_classes.append(classes["Class"])
print(list_classes)

['Accordion', 'Adhesive tape', 'Aircraft', 'Airplane', 'Alarm clock', 'Alpaca', 'Ambulance', 'Animal', 'Ant', 'Antelope', 'Apple', 'Armadillo', 'Artichoke', 'Auto part', 'Axe', 'Backpack', 'Bagel', 'Baked goods', 'Balance beam', 'Ball', 'Balloon', 'Banana', 'Band-aid', 'Banjo', 'Barge', 'Barrel', 'Baseball bat', 'Baseball glove', 'Bat (Animal)', 'Bathroom accessory', 'Bathroom cabinet', 'Bathtub', 'Beaker', 'Bear', 'Bed', 'Bee', 'Beehive', 'Beer', 'Beetle', 'Bell pepper', 'Belt', 'Bench', 'Bicycle', 'Bicycle helmet', 'Bicycle wheel', 'Bidet', 'Billboard', 'Billiard table', 'Binoculars', 'Bird', 'Blender', 'Blue jay', 'Boat', 'Bomb', 'Book', 'Bookcase', 'Boot', 'Bottle', 'Bottle opener', 'Bow and arrow', 'Bowl', 'Bowling equipment', 'Box', 'Boy', 'Brassiere', 'Bread', 'Briefcase', 'Broccoli', 'Bronze sculpture', 'Brown bear', 'Building', 'Bull', 'Burrito', 'Bus', 'Bust', 'Butterfly', 'Cabbage', 'Cabinetry', 'Cake', 'Cake stand', 'Calculator', 'Camel', 'Camera', 'Can opener', 'Canary', '

In [25]:
list_precision = []
list_recall = []
super_df = pd.DataFrame(columns=['Class', 'Precision', 'Recall'])
for classes in list_classes:
    print("Class: ", classes)
    imgs_in_db = []
    imgs_in_ground_truth = []
    for index, row in new_dataset_ground_truth_compare.iterrows():
        if classes in row['Description_x']:
            imgs_in_db.append(row['ImageID'])
        if classes in row['Description_y']:
            imgs_in_ground_truth.append(row['ImageID'])
    print("Images in DB: " + str(len(imgs_in_db)))
    print("Images in Ground Truth: " + str(len(imgs_in_ground_truth)))
    precision = 0
    recall = 0
    right_imgs = 0
    wrong_imgs = 0
    if len(imgs_in_ground_truth) == 0:
        if len(imgs_in_db) == 0:
            precision = 1
            recall = 1
            list_precision.append(precision)
            list_recall.append(recall)
            continue
        else:
            precision = 0
            recall = 0
            list_precision.append(precision)
            list_recall.append(recall)
            continue
    
    else:
        if len(imgs_in_db) == 0:
            precision = 0
            recall = 0
            list_precision.append(precision)
            list_recall.append(recall)
            continue
        else:
            for img in imgs_in_db:
                if img in imgs_in_ground_truth:
                    right_imgs += 1
                else:
                    wrong_imgs += 1
            precision = right_imgs / len(imgs_in_db)
            recall = right_imgs / len(imgs_in_ground_truth)
            list_precision.append(precision)
            list_recall.append(recall)
    super_df.loc[len(super_df)] = [classes, precision, recall]
    print("Right images: ", right_imgs)
    print("Wrong images: ", wrong_imgs)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("----------------------------------------------------")

avg_precision = sum(list_precision) / len(list_precision)
avg_recall = sum(list_recall) / len(list_recall)
print("Average Precision: ", avg_precision)
print("Average Recall: ", avg_recall)
super_df.loc[len(super_df)] = ["Average", avg_precision, avg_recall]
super_df.to_csv('../../data_files/new_precision_recall.csv', index=False)

Class:  Accordion
Images in DB: 53
Images in Ground Truth: 42
Right images:  40
Wrong images:  13
Precision:  0.7547169811320755
Recall:  0.9523809523809523
----------------------------------------------------
Class:  Adhesive tape
Images in DB: 5
Images in Ground Truth: 14
Right images:  4
Wrong images:  1
Precision:  0.8
Recall:  0.2857142857142857
----------------------------------------------------
Class:  Aircraft
Images in DB: 3
Images in Ground Truth: 1004
Right images:  2
Wrong images:  1
Precision:  0.6666666666666666
Recall:  0.00199203187250996
----------------------------------------------------
Class:  Airplane
Images in DB: 894
Images in Ground Truth: 663
Right images:  617
Wrong images:  277
Precision:  0.6901565995525727
Recall:  0.9306184012066365
----------------------------------------------------
Class:  Alarm clock
Images in DB: 1
Images in Ground Truth: 9
Right images:  1
Wrong images:  0
Precision:  1.0
Recall:  0.1111111111111111
--------------------------------